Это ноутбук с занятия по временным рядам. Здесь реализованы методы предсказания одного временного ряда из выборки по ресторанам тремя разными способами.

Но, увы, в два последних предсказания с помощью ARIMA и средней закралась ошибка и реальные тестовые данные таким образом мы бы предсказать не могли.

## Задание на понимание. 
## Срок: 15 декабря, на занятии разберем ответ.
В помеченных ниже ячейках вам надо будет поменять всего по одной строчке, чтобы получить корректно рассчитанные предсказания. 

Какой теперь метод оказался самым точным?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('../data/air_visit_data.csv.zip',)
#                  index_col='visit_date', parse_dates=True)
# df = df[df['air_store_id'] == 'air_cb7467aed805e7fe']
# df.drop('air_store_id', inplace=True, axis=1)
# df.dropna(how='any', inplace=True)
df.head(20)

In [ ]:
df['air_store_id'].nunique()

In [ ]:
df['air_store_id'].value_counts()

In [ ]:
data = pd.pivot_table(df, values='visitors', index='visit_date', columns='air_store_id')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=(255,150))
sns_heatmap = sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='OrRd')

In [ ]:
target = 'air_cb7467aed805e7fe'

In [ ]:
df = data.dropna(subset=[target], how='any')
df.shape

In [ ]:
df = df.fillna(0)

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), 
                                                    df[target],
                                                    shuffle=False,
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)

In [ ]:
model = Ridge(10)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
sns.set(font_scale=1)

fig = plt.figure(figsize=(25, 5))
plt.plot(y_test, label='real')
plt.plot(preds, color='red', label='pred')
plt.legend()
plt.xticks(rotation=70)
plt.show()

In [ ]:
score(y_test, preds)**(1/2)

In [ ]:
df[target].mean(), df[target].std()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def adf_check(time_series):
    """
    На вход получает временной ряд, выдает ADF репорт
    """
    result = adfuller(time_series)
    print('Augmented Dickey-Fuller Test:')
    labels = ['ADF Test Statistic','p-value',
              '#Lags Used','Number of Observations Used']

    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print('------------------')
        print("Сильные доказательства против нулевой гипотезы, она отвергается. \nРяд не имеет единичного корня и является стационарным.")
    else:
        print('------------------')
        print("Слабые доказательства против нулевой гипотезы, она не отвергается. \nРяд имеет единичный корень и не является стационарным.")

In [ ]:
adf_check(y_train)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig_first = plot_acf(df[target], lags=40)

In [ ]:
result = plot_pacf(df[target].dropna(), lags =40)

In [ ]:
model = ARIMA(df[target],order=(0,0,0))
results = model.fit()
print(results.summary())

In [ ]:
model = ARIMA(df[target],order=(1,0,1))
results = model.fit()
print(results.summary())

In [ ]:
model = ARIMA(df[target],order=(1,0,0))
results = model.fit()
print(results.summary())

In [ ]:
model = ARIMA(df[target],order=(2,0,2))
results = model.fit()
print(results.summary())

In [ ]:
import numpy as np
model = ARIMA(np.array(df[target]),order=(1,0,0))
results = model.fit()
print(results.summary())

In [ ]:
# Здесь ошибка!

from sklearn.metrics import mean_squared_error
 
train = y_train
test = y_test
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(1,0,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
#     print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test RMSE: %.3f' % error**(1/2))
# plot
fig = plt.figure(figsize=(25, 5))
plt.plot([x for x in test], label='real')
plt.plot(predictions, color='red', label='pred')
plt.legend()
plt.show()

In [ ]:
# И даже здесь ошибка!

score(y_test, [y_test.mean()]*len(y_test))**(1/2)